Recomendação de filmes baseada nos filmes assistidos

 ## Dependências necessárias para o funcionamento. 

In [17]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann

In [18]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [19]:
import tensorflow_recommenders as tfrs

Aqui estamos fazendo a leitura do dataset movielens.  Selecionamos as avalaições e os filmes da base.

In [20]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

In [21]:
for x in ratings.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


In [22]:
for x in movies.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


Das avalições vamos selecionar somente o filme e o Id do usuário

In [23]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

Separamos 80% dos dados para treinamento e 20% para teste.

In [24]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

Fazemos um distinct dos títulos e dos ids de usuários.

In [25]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

In [26]:
embedding_dimension = 32

In [ ]:
Criamos o sub modelo de usuário

In [27]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

Fazemos o mesmo com a torre candidata.

In [28]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

Em nossos dados de treinamento, temos pares positivos (usuário, filme). Para descobrir o quão bom é nosso modelo, precisamos comparar a pontuação de afinidade que o modelo calcula para este par com as pontuações de todos os outros candidatos possíveis: se a pontuação do par positivo for maior do que para todos os outros candidatos, nosso modelo é altamente preciso.

In [29]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)

In [ ]:
Calculo de perda

In [30]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

Definimos nosso modelo, que irá receber os 2 sub modelos que criamos.

In [31]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    user_embeddings = self.user_model(features["user_id"])
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, positive_movie_embeddings)

Instanciamos nosso modelo e ajustamos a avaliação.

In [33]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

Fazemos o embaralhamento dos dados de treino e teste.

In [34]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

Fazemos o treinamento do nosso modelo e definimos que serão 3 epocas.

In [35]:
model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 9s 700ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0013 - factorized_top_k/top_5_categorical_accuracy: 0.0095 - factorized_top_k/top_10_categorical_accuracy: 0.0203 - factorized_top_k/top_50_categorical_accuracy: 0.1001 - factorized_top_k/top_100_categorical_accuracy: 0.1771 - loss: 69885.1129 - regularization_loss: 0.0000e+00 - total_loss: 69885.1129
Epoch 2/3
10/10 [==============================] - 7s 719ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0027 - factorized_top_k/top_5_categorical_accuracy: 0.0189 - factorized_top_k/top_10_categorical_accuracy: 0.0380 - factorized_top_k/top_50_categorical_accuracy: 0.1696 - factorized_top_k/top_100_categorical_accuracy: 0.2930 - loss: 67523.3714 - regularization_loss: 0.0000e+00 - total_loss: 67523.3714
Epoch 3/3
10/10 [==============================] - 7s 712ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0037 - factorized_top_k/top_5_categorical_accuracy: 

Testamos o desempenho do nosso modelo.

In [36]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 3s 322ms/step - factorized_top_k/top_1_categorical_accuracy: 9.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0096 - factorized_top_k/top_10_categorical_accuracy: 0.0219 - factorized_top_k/top_50_categorical_accuracy: 0.1248 - factorized_top_k/top_100_categorical_accuracy: 0.2325 - loss: 31079.0635 - regularization_loss: 0.0000e+00 - total_loss: 31079.0635


{'factorized_top_k/top_1_categorical_accuracy': 0.0008999999845400453,
 'factorized_top_k/top_5_categorical_accuracy': 0.009600000455975533,
 'factorized_top_k/top_10_categorical_accuracy': 0.021900000050663948,
 'factorized_top_k/top_50_categorical_accuracy': 0.12479999661445618,
 'factorized_top_k/top_100_categorical_accuracy': 0.2325499951839447,
 'loss': 28244.771484375,
 'regularization_loss': 0,
 'total_loss': 28244.771484375}

Vamos utilizar nosso modelo para algum dos usuários da base. 

In [55]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

# Get recommendations.
userId = "42"
_, titles = index(tf.constant([userId]))
print(f"Recommendations for user {userId}: {titles[0, :1]}")

Recommendations for user 42: [b'Bridges of Madison County, The (1995)']
